In [1]:
import os 
import numpy as np
from glob import glob

## 一、获取模型信息

In [2]:
def get_edges(faces):
    """
    根据面得到相应的边
    @faces: 模型的所有面
    return: 模型的边
    """
    edge2key = dict()
    edges = []
    edges_count = 0
    for face_id, face in enumerate(faces):
        faces_edges = []
        for i in range(3):
            cur_edge = (face[i], face[(i + 1) % 3])
            faces_edges.append(cur_edge)
        for idx, edge in enumerate(faces_edges):
            edge = tuple(sorted(list(edge)))
            if edge not in edge2key:
                edge2key[edge] = edges_count
                edges_count += 1
                edges.append(list(edge))
    return edges


def parse_obje(obj_file):
    """
    解析obj文件， 获取点，边，面
    @obj_file: obj模型文件路径
    return: 模型的点，边，面信息
    """
    
    vs = []
    faces = []
    edges = []

    with open(obj_file) as f:
        for line in f:
            line = line.strip()
            splitted_line = line.split()
            if not splitted_line:
                continue
            elif splitted_line[0] == 'v':
                vs.append([float(v) for v in splitted_line[1:]])
            elif splitted_line[0] == 'f':
                try:
                    faces.append([int(c) - 1 for c in splitted_line[1:]])
                except ValueError:
                    faces.append([int(c.split('/')[0]) - 1 for c in splitted_line[1:]])                   
            elif splitted_line[0] == 'e':
                if len(splitted_line) >= 4:
                    edge_v = [int(c) - 1 for c in splitted_line[1:-1]]
                    edge_c = int(splitted_line[-1])
                    edge_v.append(edge_c)                 # class
                    edges.append(edge_v)           
            else:
                continue

    vs = np.array(vs)
    faces = np.array(faces, dtype=int)
    if len(edges) == 0:
        edges = get_edges(faces)
    edges = np.array(edges)
        
    return vs, faces, edges

## 二、error: zero_area_face

In [6]:
def compute_face_normals_and_areas(vs, faces):
    """
    计算每个面的法向量和面积
    """
    face_normals = np.cross(vs[faces[:, 1]] - vs[faces[:, 0]],
                            vs[faces[:, 2]] - vs[faces[:, 1]])
    
    # ----- deal zero face ------
    # Case1: Collinear
    zeros_idx = np.argwhere((face_normals[:,0]==0) & (face_normals[:,1]==0) & (face_normals[:,2]==0))
    # print(zeros_idx)
    # TODO: 对错误的进行调整 取平均值是否可行？
    normal_mean = np.mean(face_normals, axis=0)
    print("normal mean: ", normal_mean)
    for idx in zeros_idx:
        face_normals[idx] = normal_mean
    # Case2: ？？？
    
    face_areas = np.sqrt((face_normals ** 2).sum(axis=1))
    # print("n_faces: ", len(faces), mesh.filename)
    face_normals /= face_areas[:, np.newaxis]
    
    assert (not np.any(face_areas[:, np.newaxis] == 0)), 'has zero area face'
    face_areas *= 0.5
    return face_normals, face_areas

In [8]:
test_error_path = "/home/heygears/work/Tooth_data_prepare/model_test/error_model/train/zero_area_face"
model_list = glob(os.path.join(test_error_path, "*.obj"))
model_list

['/home/heygears/work/Tooth_data_prepare/model_test/error_model/train/zero_area_face/23RMQ_VS_SET_VSc1_Subsetup8_Mandibular.obj',
 '/home/heygears/work/Tooth_data_prepare/model_test/error_model/train/zero_area_face/8M2JR_VS_SET_VSc3_Subsetup16_Maxillar.obj',
 '/home/heygears/work/Tooth_data_prepare/model_test/error_model/train/zero_area_face/6DS6F_VS_SET_VSc2_Subsetup12_Mandibular.obj',
 '/home/heygears/work/Tooth_data_prepare/model_test/error_model/train/zero_area_face/632WS_VS_SET_VSc1_Subsetup15_Maxillar.obj',
 '/home/heygears/work/Tooth_data_prepare/model_test/error_model/train/zero_area_face/XZMTU_VS_SET_VSc2_Subsetup13_Maxillar.obj']

In [7]:
for i, model_path in enumerate(model_list):
    obj_vs, obj_faces, obj_edges = parse_obje(model_path)
    compute_face_normals_and_areas(obj_vs, obj_faces)

normal mean:  [ 2.18713936e-18 -1.69377967e-17  1.09423581e-16]
normal mean:  [ 1.80072970e-17  2.15318215e-17 -2.11120010e-16]
normal mean:  [ 6.73738842e-17  6.87720714e-17 -1.16990861e-15]
normal mean:  [ 1.07349459e-17 -9.75163093e-17  8.71480665e-16]
normal mean:  [ 3.78778606e-17  2.71426978e-17 -7.75668418e-16]


### 测试单个
#### 1、下采样后模型

In [6]:
obj_vs, obj_faces, obj_edges = parse_obje(model_list[0])
len(obj_faces)

5000

In [7]:
face_normals = np.cross(obj_vs[obj_faces[:, 1]] - obj_vs[obj_faces[:, 0]],
                            obj_vs[obj_faces[:, 2]] - obj_vs[obj_faces[:, 0]])

In [8]:
face_normals

array([[ 1.71525820e+00,  1.39445200e+00,  7.56778633e-02],
       [ 1.01242707e+00,  1.22013991e+00, -1.17839972e-01],
       [ 1.86599495e+00, -1.24528945e+00,  1.51754871e+00],
       ...,
       [ 3.06748695e-03,  3.87738707e-04, -2.55378255e+00],
       [-3.29822436e-04,  2.90588371e-03, -2.13824729e+00],
       [-2.48862571e-03, -1.81972596e-03, -5.09452535e+00]])

In [9]:
def pts_to_vtk(gum_line_pts, save_path="./test.vtk"):
    """
    将牙龈点pts格式转为vtk格式
    @pts: 点集 [[x, y, z], [x, y, z], ...]
    @save_path: 保存路径
    return: None
    """
    import vtkplotter as vtkp
    vtk_point = vtkp.Points(gum_line_pts.reshape(-1, 3))
    vtkp.write(vtk_point, save_path, binary=False)
    print("vtk file is saved in ", save_path)

In [10]:
error_idx = np.argwhere((face_normals[:,0]==0) & (face_normals[:,1]==0) & (face_normals[:,2]==0))
error_idx

array([[  98],
       [1402]])

In [11]:
pts = []
for idx in error_idx:
    for vertex_id in obj_faces[idx][0]:
        pts.append(obj_vs[vertex_id])
pts_to_vtk(np.asarray(pts))

vtk file is saved in  ./test.vtk


In [12]:
face_normals[51]

array([ 0.00849412,  0.3082446 , -0.94659179])

In [13]:
obj_faces[51]

array([  45, 1712, 1525])

In [14]:
print(obj_vs[67], obj_vs[68], obj_vs[69])

[-10.064562  23.082346   5.303358] [ 30.175774 -10.169944   9.648704] [-25.77187    7.551851   7.875569]


In [15]:
face_areas = np.sqrt((face_normals ** 2).sum(axis=1))
face_areas

array([2.21186216, 1.58985415, 2.70843812, ..., 2.55378442, 2.13824929,
       5.09452629])

In [16]:
0 in face_areas

True

#### 利用vedo库将stl转obj  

In [17]:
origin_vs, origin_faces, origin_edges = parse_obje("/home/heygears/work/Tooth_data_prepare/model_test/origin_mesh_obj/459650_1_L_19.obj")

In [18]:
origin_face_normals = np.cross(origin_vs[origin_faces[:, 1]] - origin_vs[origin_faces[:, 0]],
                            origin_vs[origin_faces[:, 2]] - origin_vs[origin_faces[:, 0]])

In [19]:
np.argwhere((origin_face_normals[:,0]==0) & (origin_face_normals[:,1]==0) & (origin_face_normals[:,2]==0))

array([[ 751],
       [ 762],
       [ 764],
       [1029]])

In [20]:
print(len(origin_vs), len(origin_faces), len(origin_edges))

142828 285648 428472


#### 利用hgapi将stl转obj模型

In [21]:
import hgapi
stl_handler = hgapi.STLFileHandler()
mesh = hgapi.Mesh()
file_name = "/home/heygears/work/Tooth_data_prepare/model_test/originMesh/459650_1_L_19.stl"
stl_handler.Read(file_name, mesh)

obj_handler = hgapi.OBJFileHandler()
obj_handler.Write("./test.obj", mesh)

True

In [22]:
origin_vs, origin_faces, origin_edges = parse_obje("./test.obj")

In [23]:
origin_face_normals = np.cross(origin_vs[origin_faces[:, 1]] - origin_vs[origin_faces[:, 0]],
                            origin_vs[origin_faces[:, 2]] - origin_vs[origin_faces[:, 0]])
np.argwhere((origin_face_normals[:,0]==0) & (origin_face_normals[:,1]==0) & (origin_face_normals[:,2]==0))

array([[ 751],
       [ 762],
       [ 764],
       [1029]])

#### 原始stl模型

In [24]:
stl_handler = hgapi.STLFileHandler()
mesh = hgapi.Mesh()
file_name = "/home/heygears/work/Tooth_data_prepare/model_test/originMesh/459650_1_L_19.stl"
stl_handler.Read(file_name, mesh)

mesh.GenerateFaceNormal(bGenerateFaceArea=True)
normal = mesh.GetFaceNormal()
area = mesh.GetFaceArea()
face_areas = np.array(list(area))
print(len(face_areas), np.argwhere(face_areas == 0))

285648 []


## 三、index is negative

In [25]:
test_error_path = "/home/heygears/work/Tooth_data_prepare/model_test/error_model/train/index_is_negative/"
model_list = glob(os.path.join(test_error_path, "*.obj"))
len(model_list)

1

In [26]:
obj_vs, obj_faces, obj_edges = parse_obje(model_list[0])

In [27]:
obj_faces

array([[1016, 1243,  285],
       [ 963, 1035,  204],
       [ 821,  215,  833],
       ...,
       [1386, 1683,   52],
       [1683, 1445,  211],
       [1683,  158, 1445]])

In [30]:
np.argwhere(obj_faces < 0)

array([], shape=(0, 2), dtype=int64)

## 四、index out of range

In [31]:
test_error_path = "/home/heygears/work/Tooth_data_prepare/model_test/error_model/train/index_out_of_range/"
model_list = glob(os.path.join(test_error_path, "*.obj"))
len(model_list)

9

In [39]:
for i, model_path in enumerate(model_list):
    obj_vs, obj_faces, obj_edges = parse_obje(model_path)
    print(i, len(obj_vs), len(obj_faces), len(obj_edges))
    check_id = np.argwhere(obj_faces<0)
    print(check_id)

0 2154 5000 7500
[]
1 2074 5000 7500
[]
2 2132 5000 7500
[]
3 2026 5000 7500
[]
4 2042 5000 7500
[]
5 2060 5000 7500
[]
6 2142 5000 7500
[]
7 2152 5000 7500
[]
8 2028 5000 7500
[]


## 数据特征提取

In [25]:
def pad(input_arr, target_length, val=0, dim=1):
    shp = input_arr.shape
    print("shp: ", shp, len(shp))
    npad = [(0, 0) for _ in range(len(shp))]
    npad[dim] = (0, target_length - shp[dim])
    return np.pad(input_arr, pad_width=npad, mode='constant', constant_values=val)


In [22]:
def read_seg(seg):
    seg_labels = np.loadtxt(open(seg, 'r'), dtype='float64')
    return seg_labels

In [23]:
label = read_seg("/home/heygears/work/github/MeshCNN/datasets/tooth_seg/seg/1PT7E_VS_SET_VSc2_Subsetup_Retainer_Maxillar.eseg")
len(label)

7500

In [26]:
ninput_edges = 7600
pad_label = pad(label, ninput_edges, val=-1, dim=0)
len(pad_label)

shp:  (7500,) 1


7600

In [29]:
a = [1, 2, 3, 4, 5]

np.pad(a, (0, 3), 'constant', constant_values=(4))

array([1, 2, 3, 4, 5, 4, 4, 4])

In [30]:
np.pad(a, (2, 3), 'edge')

array([1, 1, 1, 2, 3, 4, 5, 5, 5, 5])